In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
import custom_funcs as cf

In this notebook, I will train plain vanilla RandomForestRegressor models for each of the protease drug resistance values, and pickle them to disk.

In [ ]:
drugs = ['FPV', 'ATV', 'IDV', 'LPV', 'NFV', 'SQV', 'TPV', 'DRV',]
drug = drugs[1]
protein = 'protease'

data, feat_cols = cf.get_cleaned_data(protein, drug)

# Just checking:
cf.test_data_integrity(data)

## Now, let's do data transformations.
data_numeric = cf.to_numeric_rep(data, feat_cols, rep='mw')

# Finally, split the data into a training set, and test set.
X, Y, X_train, X_test, Y_train, Y_test = cf.to_train_test_split(data_numeric, feat_cols, drug, test_size=0.3)
# sscv = ShuffleSplit(n=len(X_train), n_iter=3, test_size=0.3)

In [ ]:
mdl = RandomForestRegressor(n_estimators=2000, min_samples_leaf=1, n_jobs=-1, )
mdl.fit(X, Y)

In [ ]:
import numpy as np

def pred_intervals(model, X, percentile=95):
    """
    From http://blog.datadive.net/prediction-intervals-for-random-forests/
    
    Computes prediction intervals to compute uncertainty.
    """
    err_down = []
    err_up = []
    avg_preds = []
    for row in range(len(X)):
        # print(x)
        preds = []
        for est in model.estimators_:
            preds.append(est.predict(X[row].reshape(1, -1))[0])
        err_down.append(np.percentile(preds, (100 - percentile) / 2. ))
        err_up.append(np.percentile(preds, 100 - (100 - percentile) / 2.))
        avg_preds.append(np.mean(preds))
    return err_down, err_up, avg_preds

In [ ]:
%%time
err_down, err_up, avg_preds = pred_intervals(mdl, X_test.values)
err_down, err_up, avg_preds

In [ ]:
yerr_down = np.array(avg_preds) - np.array(err_down)
yerr_up = np.array(err_up) - np.array(avg_preds)

In [ ]:
for i in range(len(err_down)):
    assert err_down[i] < avg_preds[i]
    assert err_up[i] > avg_preds[i]

In [ ]:
yerr_down

In [ ]:
preds = mdl.predict(X_test.values)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure(figsize=(16, 5))
plt.errorbar(Y_test.values, avg_preds, yerr=[yerr_down, yerr_up], marker='o', ls='none', alpha=0.3)
# plt.xlim(-1.5, 3)
# plt.ylim(-1.5, 3)
plt.hlines(y=[0, np.log10(3)], xmin=min(Y_test.values), xmax=max(Y_test.values))

In [ ]:
joblib.dump(mdl, '../models/base/{drug}/{drug}.pkl'.format(drug=drug))